In [2]:
import $file.hw8stdlib
import hw8stdlib._

import $file.$        

import hw8stdlib._

In [3]:
type Parser[S,D] = List[S] => List[(D, List[S])]

def char(c : Char) : Parser[Char,Char] = 
(ss : List[Char]) => ss match {
    case Empty      => Empty
    case Cons(s,ss) => char_eq(s,c) match {
        case True  => singleton((s, ss))
        case False => Empty
    }
}

def success[S, D](x : D) : Parser[S, D] = 
  (ss : List[S]) => singleton((x, ss))

def failure[S, D]() : Parser[S,D] = (ss : List[S]) => Empty

def choose[S, D](p : Parser[S, D], q : Parser[S, D]) : Parser[S, D] = (ss : List[S]) =>
{
    val p_res = p(ss)
    val q_res = q(ss)
    append(p_res, q_res)
} 

defined type Parser
defined function char
defined function success
defined function failure
defined function choose

# Homework 8

In this week's homework we will build up more interesting parsers by creating some new combinators using `bind`

## RunParser

`runParser` is a useful function for running a parser on an input. You will see it used in some of the examples below.

In [4]:
def runParser[D](p : Parser[Char, D], s : String) : Maybe[D] = p(string_to_list(s)) match {
    case Empty                   => None
    case Cons((x, Empty), Empty) => Just(x)
    case _                       => None
}

defined function runParser

# Problem 1 - `satisfies` (10 points)

For the following type:

$$
\text{satisfies } : \underbrace{(S \rightarrow \mathbb{B})}_{\text{predicate}} \rightarrow \textbf{Parser } S\ \textbf{Char}
$$

Implement `satisfies` such that it successfully parses a symbol that the predicate is true for and fails if the predicate is false. 

***hint*** The implementation of `char` above may be helpful as a starting point.

In [5]:
def satisfies[S](p : (S => Bool)) : Parser[S, S] = (xs : List[S]) => xs match{
    case Empty => Empty
    case Cons(s, ss) => p(s) match{
        case True => singleton((s, ss))
        case False => Empty
    }
} 


defined function satisfies

In [6]:
// Tests empty input
def pt(x: Char): Bool = True
def pf(x: Char): Bool = False
assert(runParser(satisfies(pt), "") == None)
assert(runParser(satisfies(pf), "") == None)
passed(5)


*** Tests Passed (5 points) ***


defined function pt
defined function pf

In [7]:
// Tests all true or false
assert(satisfies(pt)(string_to_list("a")) == Cons(('a',Empty),Empty))
assert(satisfies(pt)(string_to_list("abc")) == Cons(('a',Cons('b',Cons('c',Empty))),Empty))
assert(satisfies(pf)(string_to_list("abc")) == Empty)
passed(3)


*** Tests Passed (3 points) ***


In [8]:
// Tests specific true or false
assert(satisfies[Char](char_eq(_, '0'))(string_to_list("0bc")) == Cons(('0',Cons('b',Cons('c',Empty))),Empty))
assert(satisfies[Char](char_eq(_, '0'))(string_to_list("abc")) == Empty)
passed(4)


*** Tests Passed (4 points) ***


# Bind - Sequencing our Parses

`bind` allows us to sequence parses. It can be a little confusing at first but with some practice it becomes very intuitive to use.

The type for bind is: 


$$
\textbf{bind } : \underbrace{\textbf{Parser } S\ D_1\ }_{p_1\text{ - First Parser}}\rightarrow \underbrace{(D_1 \rightarrow \textbf{Parser } S\ D_2\ )}_{p_2\text{ - Generator for Second Parser}} \rightarrow \underbrace{\textbf{Parser } S\ D_2}_{p_1p_2\text{ - Composite Parser} }
$$

Let's break down each part of this signature:

* $p_1$ is the first parser we will use on an input. It will give us a parsed result of type $D_1$. We will then feed this result into the next part.
* $p_2$ is a generator for a parser which takes, as an argument something of type $D_1$, and returns a parser which gives results of type $D_2$. `bind` takes the result of running the first parser $p_1$ on some input and then feeds that result into this function, producing the output type for `bind`- $\textbf{Parser } S\ D_2$

Here is the implementation of `bind`. Don't worry too much about how it works. We will get an intuition for it through some examples.

In [9]:
def bind[S, D, E](p : Parser[S,D], q : (D => Parser[S,E]) ) : Parser[S,E] = 
  (ss : List[S]) => {
      val join = (res : (D, List[S])) => res match {case (d, ss2) => q(d)(ss2)}
      concatMap(join, p(ss))
  } 

defined function bind

## Example 1 - $ a $ then $b$

In this example we will write a parser which recognizes the sequence of characters `"ab"`. We can begin by defining the parser for the first letter, `'a'`. For this we just need to use `char`:

In [10]:
def parseLetterA : Parser[Char,Char] = char('a')

defined function parseLetterA

`parseLetterA` will be the first argument to `bind`. The next thing we need is a function of type:  $D_1 \rightarrow \textbf{Parser } S\ D_2$

We will start by defining the parser for just the letter B:

In [11]:
def parseLetterB : Parser[Char, Char] = char('b')

defined function parseLetterB

We would like our parse result to be a list of characters that contains "ab"(which makes $D_2 = \textbf{List Char}$). We also know that the result of the first parser is a character(This means $D_1 = \textbf{Char}$) This specializes this type to: $\textbf{Char} \rightarrow \textbf{Parser } S\ (\textbf{List Char})$. 


We will say that this is a parser that takes a letter and then parses a `'b'`. After doing this it combines them into a list of characters. 

In [12]:
def parseLetterThenB(x : Char) : Parser[Char, List[Char]] =
    bind(
        parseLetterB,
        (letterb : Char) => success(Cons(x, Cons(letterb, Empty)))
    )

defined function parseLetterThenB

Above we provided bind a parser for the letter `'b'` and a lambda-function which takes the result of that parse and combines it with the letter `x` that was passed in as an argument.

Now that we have this we can combine it all into our final parser for `"ab"`:

In [13]:
def parseAThenB : Parser[Char, List[Char]] =
    bind(parseLetterA, parseLetterThenB)

defined function parseAThenB

If we test this we will notice it behaves as we would expect:

In [14]:
parseAThenB(string_to_list("ab"))

res13: List[(List[Char], List[Char])] = Cons(
  (Cons('a', Cons('b', Empty)), Empty),
  Empty
)

## Example 2 - $abc$

Writing parsers as a bunch of small functions like this often becomes cumbersome so we prefer to write them as a single definition using several calls to `bind` at once. Here is an example of this for the string `"abc"`:

In [15]:
def parseABC : Parser[Char, List[Char]]  = bind(char('a'), 
                           (p_a : Char) => bind(char('b'), 
                           (p_b : Char) => bind(char('c'),
                           (p_c : Char) => success(Cons(p_a, Cons(p_b, Cons(p_c, Empty)))))))

defined function parseABC

Here is a quick proof that it works:

In [16]:
runParser(parseABC, "abc")
runParser(parseABC, "xyz")

res15_0: Maybe[List[Char]] = Just(Cons('a', Cons('b', Cons('c', Empty))))
res15_1: Maybe[List[Char]] = None

To read the function above just take each call to `bind` to mean that we are calling a parser and using its result. So on the second line we have $p_a$ which is the character that resulted from parsing $\text{char}('a')$. In the next bind we have access to both $p_a$ and $p_b$. In the third we have all three results and can combine them into the list we wanted. We need to wrap this list up in `success` so that it has the necessary type($\textbf{Parser Char (List Char)}$)

***Note on Layout*** We strongly recommend laying out calls to bind on multiple lines for readability. It need not be in the same manner as we have done here though. For instance the lambda terms could have been on the same line as their bind call. Any kind of spacing that helps read the code is a good idea here.

## `string` - A New Combinator

The two examples above are examples of a more general class of parsers where we wish to parse some specific combination of characters, or a string. We can write a special parser generator which takes a string and creates a parser for that string:

In [17]:
def stringL(ss : List[Char]) : Parser[Char, List[Char]] = ss match {
    case Empty      => success(Empty)
    case Cons(s,ss) =>        bind(char(s), 
         (c : Char)        => bind(stringL(ss),
         (cs : List[Char]) => success(Cons(c,cs)) ))
}

def string(str : String) : Parser[Char, List[Char]] = stringL(string_to_list(str))

defined function stringL
defined function string

In [18]:
runParser(string("abc"), "abc")

res17: Maybe[List[Char]] = Just(Cons('a', Cons('b', Cons('c', Empty))))

# Problem 2 - Lettuce Keywords (10 points)

Write a parser, using `string` and `choice` which accepts the strings `"let"`, `"in"`, `"function"`, and `"rec"`

In [19]:
def parseKeywords : Parser[Char, List[Char]] = choose(string("let"), 
                                                      choose(string("in"),
                                                            choose(string("function"),
                                                                  string("rec"))))
    

defined function parseKeywords

If your solution is correct the following examples should be parsed successfully:

In [20]:
assert(runParser(parseKeywords, "let") == Just(string_to_list("let")))
assert(runParser(parseKeywords, "rec") == Just(string_to_list("rec")))
assert(runParser(parseKeywords, "function") == Just(string_to_list("function")))
assert(runParser(parseKeywords, "in") == Just(string_to_list("in")))
passed(5)


*** Tests Passed (5 points) ***


and the following examples should fail:

In [21]:
assert(runParser(parseKeywords, "x") == None)
assert(runParser(parseKeywords, "or") == None)
assert(runParser(parseKeywords, "functio") == None)
assert(runParser(parseKeywords, "ni") == None)
passed(5)


*** Tests Passed (5 points) ***


# Problem 3 - `mapParser` (10 points)

One thing we may want to do to a parser is apply a function to its results. A function that does this could have the type:

$$
\text{mapParser} : (A \rightarrow B) \rightarrow \textbf{Parser } S\ A\ \rightarrow \textbf{Parser } S\ B
$$

We do something very much like this by applying cons to the result of our parsers above. Using `bind` write the `mapParser` function below:

In [22]:
def mapParser[S,A,B](f : (A => B), p : Parser[S,A]) : Parser[S,B] = 
bind(p, (l: A) => success[S, B](f(l)))
    

defined function mapParser

In [23]:
// Tests Empty input
assert(runParser(mapParser(list_to_string, string("")), "") == Just(""))
passed(3)


*** Tests Passed (3 points) ***


In [24]:
// Tests non empty success
assert(runParser(mapParser(list_to_string, string("abc")), "abc") == Just("abc"))
assert(runParser(mapParser(reverse[Char], string("abcd")), "abcd") == Just(string_to_list("dcba")))
passed(4)


*** Tests Passed (4 points) ***


In [25]:
// Tests failure
assert(runParser(mapParser(list_to_string, string("abc")), "xyz") == None)
passed(3)


*** Tests Passed (3 points) ***
